In [ ]:
!pip install fake_useragent

In [ ]:
# для скачивания текстов
import requests
from fake_useragent import UserAgent
import time
import random
from bs4 import BeautifulSoup

# для отслеживания всяких прогрессов
from tqdm import tqdm

Подключаемся к сайту http://tolstoy-lit.ru/tolstoy/proza/index.htm

In [ ]:
session = requests.session()

In [ ]:
ua = UserAgent()

In [ ]:
url = "http://tolstoy-lit.ru/tolstoy/proza/index.htm"
headers = {'User-Agent': ua.random}

response = session.get(url, headers=headers)
response

С помощью функции get_the_books(url) по заданной ссылке находим все ссылки с произведениями

In [ ]:
def get_the_books(url):
    headers = {'User-Agent': ua.random}
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    links_texts = []
    for x in soup.find_all('a'):
        # это два условия, по которым находятся настоящие ссылки на произведения
        # (иначе иногда попадает всякий мусор)
        if ('tolstoy' in x.attrs['href']) and (x.attrs['href'][0:4] != 'http'):
            links_texts.append('http://tolstoy-lit.ru' + x.attrs['href'])
    return links_texts

С помощью функции get_text_page(url) по заданной ссылке записываем текст для этой страницы

In [ ]:
def get_text_page(url):
    fin_t = []
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.content
    soup = BeautifulSoup(page, 'html.parser')
    text_p = soup.find_all('p', {'class': 'tab'})
    for t in text_p:
        # немного предобратавыем текст
        t = t.text
        # удаляем всякие дополнительные символы и лишние получившиеся пробелы
        t = t.replace('\n', ' ')
        t = t.replace('\r', ' ')
        t = ' '.join(t.split())
        fin_t.append(t)
        time.sleep(random.uniform(0.01, 0.59))
    return ' '.join(fin_t)

С помощью функции get_whole_text(book_url) по заданной ссылке на книгу проходимся по всем главам/частям книги (которые представляют собой одну страницу) и добавляем скачанный текст в одну строку

In [ ]:
def get_whole_text(book_url):
    whole_text = []
    req = session.get(book_url, headers={'User-Agent': ua.random})
    page = req.content
    soup = BeautifulSoup(page, 'html.parser')
    try:
        links = soup.find_all('a')
        # здесь будут уникальные ссылки на части книги. много где они повторяются
        unique_li = [book_url] 
        for l in links:
            li = 'http://tolstoy-lit.ru' + l.attrs['href']
            if li not in unique_li:
                # главы и части книги обозначены либо словом ГЛАВА, либо цифрой
                if ('Глава' in l.text) or (re.fullmatch(r'.*\d.*', l.text)):
                    unique_li.append(li)
        for i in range(len(unique_li)):
            whole_text.append(get_text_page(unique_li[i]))
            time.sleep(random.uniform(1.9, 3.9))
    # в случае если нет частей или глав
    except:
        whole_text.append(get_text_page(book_url))
    return ' '.join(whole_text)

С помощью get_everything(url) по исходной ссылке сначала находим ссылки на все книги, а затем для первых 20 книг (иначе работает сто миллионов лет и модель дальше все равно ломается от объема) находим их тексты

Отслеживаем прогресс, чтобы понимать, насколько все (не)плохо!

In [ ]:
def get_everything(url):
    texts = []
    books = get_the_books(url)
    # записываем все в файлик, чтобы сохранилось
    with open('texts_parsed.txt', 'a+') as file:
        for i in tqdm(range(0, 20)):
            print(books[i])
            text = get_whole_text(books[i])
            texts.append(text)
            # разделяем тексты \n
            file.write('\n'.join(texts))
            time.sleep(random.uniform(1.9, 3.9))

In [ ]:
get_everything('http://tolstoy-lit.ru/tolstoy/proza/index.htm')

Скачиваем файлик себе

In [ ]:
from google.colab import files
files.download('texts_parsed.txt')